In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier,VotingClassifier,BaggingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier,Pool


pd.pandas.set_option('display.max_columns', None)
import os
import datetime
import gc
import glob
import itertools

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [3]:
df.shape

(617, 58)

In [5]:
df.drop('Id',axis = 1, inplace = True)

In [7]:
df['EJ'] = df['EJ'].map({'B': 0, 'A': 1})

In [8]:
df.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,0,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,1,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,0,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,0,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,0,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [11]:
scalar = MinMaxScaler()
df_scaled = scalar.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns = df.columns)

In [12]:
df_scaled

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0.021082,0.102347,0.000000,0.030632,0.000000,0.000000,0.000000,0.180337,0.002958,0.047364,0.654545,0.043240,0.440929,0.004312,0.0,0.015372,0.098469,0.000000,0.021656,0.091892,0.000000,0.000000,0.000000,0.241834,0.507476,0.305651,0.125554,0.000000,0.238606,0.029338,0.234167,0.412931,0.032880,0.000000,0.147697,0.026299,0.094302,0.041505,0.022225,0.0,0.245682,0.000000,0.000000,0.001939,0.006316,0.052697,0.000000,0.051720,0.000998,0.000866,0.054959,0.000000,0.013846,0.176983,0.362261,0.005425,1.0
1,0.010541,0.027589,0.000000,0.053864,0.000000,0.078048,0.000000,0.284500,0.000000,0.074042,0.490909,0.034915,0.039100,0.000000,0.0,0.007873,0.078406,0.044646,0.027807,0.127928,0.002077,0.353002,0.057157,0.253295,0.529577,0.314082,0.068953,0.000000,0.316354,0.050913,0.207033,0.552387,0.000000,0.000000,0.108335,0.000000,0.031732,0.030797,0.000000,1.0,1.000000,0.017144,0.007454,0.003189,0.000000,0.036862,0.209978,0.000000,0.000000,0.016014,0.039418,0.000000,0.194527,0.274495,0.164135,1.000000,0.0
2,0.063949,0.085715,0.000000,0.046519,0.000000,0.160575,0.000000,0.265013,0.000000,0.067011,0.854545,0.023597,0.634957,0.002405,0.0,0.008882,0.081244,0.102545,0.024273,0.149550,0.000000,0.212468,0.100648,0.181089,0.250979,0.314187,0.138061,0.000000,0.166220,0.060493,0.174504,0.384532,0.007959,0.000000,0.237922,0.032069,0.435754,0.043532,0.008790,0.0,1.000000,0.000000,0.000240,0.071672,0.005353,0.047831,0.249274,0.054706,0.000385,0.036139,0.250424,0.011229,0.095035,0.258994,0.180218,0.008910,0.0
3,0.028110,0.127285,0.019180,0.117854,0.000000,0.079462,0.000000,0.215235,0.000000,0.048203,0.709091,0.069197,0.028310,0.003406,0.0,0.001196,0.137796,0.106158,0.009163,0.117117,0.011426,0.190830,0.107542,0.121490,0.252209,0.198449,0.077686,0.000000,0.246649,0.080483,0.193819,0.319033,0.016423,0.000000,0.337839,0.027320,0.195531,0.515865,0.014364,0.0,0.253346,0.000000,0.004232,0.017124,0.004807,0.066374,0.349223,0.043183,0.000000,0.006925,0.109565,0.006882,0.014475,0.425143,0.470820,0.007039,0.0
4,0.049192,0.124246,0.000000,0.017417,0.000000,0.086300,0.002841,0.000000,0.069008,0.078555,0.727273,0.105942,0.432218,0.033614,0.0,0.030861,0.091589,0.080729,0.149792,0.100901,0.000000,0.210044,0.070588,0.143266,0.476331,0.330142,0.079413,0.000000,0.163539,0.038109,0.134519,0.348520,0.007062,0.000000,0.054508,0.026927,0.177654,0.040524,0.003788,0.0,1.000000,0.013672,0.007282,0.007135,0.002520,0.103309,0.312489,0.057926,0.038597,0.001731,0.093459,0.051588,0.059198,0.500829,0.185841,0.004345,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0.011244,0.103085,0.021132,0.010101,0.028583,0.074511,0.005030,0.144730,0.001076,0.047970,0.581818,0.039972,0.075605,0.001754,0.0,0.012780,0.131014,0.052078,0.021299,0.048649,0.004155,0.211761,0.103776,0.193696,0.000000,0.073013,0.154750,0.000000,0.396783,0.118051,0.255065,0.382840,0.000000,0.028827,0.209200,0.034307,0.059218,0.010296,0.000000,1.0,0.445619,0.000000,0.009541,0.007335,0.000000,0.110150,0.195114,0.000000,0.000614,0.000000,0.036945,0.101448,0.056218,0.211871,0.358883,1.000000,0.0
613,0.058327,0.184921,0.000000,0.069139,0.045868,0.140533,0.000030,0.259345,0.001742,0.077101,0.890909,0.089554,1.000000,0.002532,0.0,0.021873,0.151657,0.101363,0.030788,0.158559,0.000000,0.232894,0.102569,0.417192,0.462989,0.181903,0.058718,0.000000,0.388740,0.133847,0.357183,0.361450,0.003984,0.000000,0.114767,0.044984,0.025922,0.051137,0.003216,0.0,1.000000,0.036824,0.068308,0.020446,0.003656,0.119279,0.227063,0.072953,0.000599,0.011469,0.241452,0.297867,0.021368,0.281649,0.651146,0.006562,0.0
614,0.056922,0.079539,0.024625,0.083174,0.010930,0.196180,0.000000,0.337644,0.000000,0.08

In [13]:
# Separate majority (class 0) and minority (class 1) samples
majority_class = 0
minority_class = 1

df_majority = df_scaled[df_scaled['Class'] == majority_class]
df_minority = df_scaled[df_scaled['Class'] == minority_class]

# Determine the number of samples to be undersampled (match minority class count)
num_minority_samples = df_minority.shape[0]

# Sample randomly from majority class to match the number of minority samples
df_majority_undersampled = df_majority.sample(n=num_minority_samples, random_state=42)

# Concatenate the undersampled majority class with the minority class to create the balanced DataFrame
df_balanced = pd.concat([df_majority_undersampled, df_minority], axis=0)

In [15]:
df = df_balanced

In [16]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
y_pred = model.predict(X_test)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [21]:
print(f'xgb-{accuracy}-{precision}-{recall}-{f1}')

xgb-0.8923076923076924-0.9354838709677419-0.8529411764705882-0.8923076923076922
